### data

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

R_flat = pd.read_csv("result/result_R.csv")

I_len = R_flat["generator"].nunique()
T_len = R_flat["time"].nunique() 
S_len = R_flat["scenario"].nunique()

R = R_flat.pivot_table(
    values="value",
    index="generator",
    columns=["time", "scenario"]
).to_numpy()
R = R.reshape(I_len, T_len, S_len)

I, T, S = R.shape

price_q = pd.read_csv(
    "/Users/jangseohyun/Documents/workspace/symply/DER/DATA_price.csv"
)
price_q["Time"] = pd.to_datetime(price_q["Time"], format="%Y-%m-%d %H:%M")

price_q["Hour"] = price_q["Time"].dt.floor("h")
price_h = price_q.groupby("Hour").mean(numeric_only=True)

price = price_h.iloc[: S * T]

P_DA = np.array(
    [sum(price["Price"].iloc[t + s * T] for s in range(S)) / S * 1.2 for t in range(T)]
)
P_RT = np.array([[price["Price"].iloc[t + s * T] for s in range(S)] for t in range(T)])
P_PN = np.array(
    [sum(price["Price"].iloc[t + s * T] for s in range(S)) / S * 2 for t in range(T)]
)

only_profit = pd.read_csv('result/result_only_profit.csv').values

### settlement model

In [20]:
set = gp.Model("Settlement")
# set.setParam('TimeLimit', 3600)
# set.setParam("PoolSolutions", 15)
# set.setParam("PoolSearchMode", 2)
# set.setParam("PoolGap", 0.05)
set.setParam("MIPGap", 1e-3)
set.setParam("Heuristics", 0.5)


x = set.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
y_plus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
y_minus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
M_y = max(R[i, t, s] for i in range(I) for t in range(T) for s in range(S))
z_y = set.addVars(I, T, S, vtype=GRB.BINARY, name="z_y")

d = set.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="d")
d_given = set.addVars(I, S, vtype=GRB.CONTINUOUS, name="d_given")
d_received = set.addVars(I, S, vtype=GRB.CONTINUOUS, name="d_received")
d_max_H = set.addVars(T, S, vtype=GRB.CONTINUOUS, name="d_max_H")
d_plus = set.addVars(I, S, vtype=GRB.CONTINUOUS, lb=0, name="d_plus")
d_minus = set.addVars(I, S, vtype=GRB.CONTINUOUS, lb=0, name="d_minus")

e_plus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_plus")
e_minus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_minus")
M_e = max(sum(R[i, t, s] for i in range(I)) for t in range(T) for s in range(S))
z_e = set.addVars(T, S, vtype=GRB.BINARY, name="z_e")

profit_increase = set.addVars(I, vtype=GRB.CONTINUOUS, lb=0, name="profit_increase")

set.update()

# obj. func.
obj = (
    gp.quicksum(P_DA[t] * x[i, t] for i in range(I) for t in range(T))
    + gp.quicksum(
        1 / S * (P_RT[t, s] * e_plus[i, t, s] - P_PN[t] * e_minus[i, t, s])
        for i in range(I)
        for t in range(T)
        for s in range(S)
    )
    - gp.quicksum((d_plus[i, s] + d_minus[i, s]) for i in range(I) for s in range(S))
)

set.setObjective(obj, GRB.MAXIMIZE)

Set parameter MIPGap to value 0.001
Set parameter Heuristics to value 0.5


In [21]:
# x로 y+, y- 결정 지어주기
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(R[i, t, s] - x[i, t] == y_plus[i, t, s] - y_minus[i, t, s])

# y+, y- 둘 중 하나는 0이어야함
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(y_plus[i, t, s] <= M_y * z_y[i, t, s])
            set.addConstr(y_minus[i, t, s] <= M_y * (1 - z_y[i, t, s]))

# rt 조건
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(y_plus[i, t, s] <= R[i, t, s])

# sum e+, sum e- 결정 지어주기
for t in range(T):
    for s in range(S):
        set.addConstr(
            gp.quicksum(R[i, t, s] for i in range(I))
            - gp.quicksum(x[i, t] for i in range(I))
            == gp.quicksum(e_plus[i, t, s] for i in range(I))
            - gp.quicksum(e_minus[i, t, s] for i in range(I))
        )

# sum e+, sum e- 둘 중 하나는 0이어야 함
for t in range(T):
    for s in range(S):
        set.addConstr(gp.quicksum(e_plus[i, t, s] for i in range(I)) <= M_e * z_e[t, s])
        set.addConstr(
            gp.quicksum(e_minus[i, t, s] for i in range(I)) <= M_e * (1 - z_e[t, s])
        )

# e+, e- 정의
for i in range(I):
    for t in range(T):
        for s in range(S):
            # e_plus: 초과량에서 준 양을 뺀 것
            set.addConstr(
                e_plus[i, t, s]
                == y_plus[i, t, s]
                - gp.quicksum(d[i, j, t, s] for j in range(I) if j != i)
            )
            # e_minus: 부족량에서 받은 양을 뺀 것
            set.addConstr(
                e_minus[i, t, s]
                == y_minus[i, t, s]
                - gp.quicksum(d[j, i, t, s] for j in range(I) if j != i)
            )

# sum e+ <= sum R
for t in range(T):
    for s in range(S):
        set.addConstr(
            gp.quicksum(e_plus[i, t, s] for i in range(I))
            <= gp.quicksum(R[i, t, s] for i in range(I))
        )

# 전력 이동 제한
for i in range(I):
    for t in range(T):
        for s in range(S):
            # 본인이 주는 양은 자신의 y+을 넘을 수 없음
            set.addConstr(
                gp.quicksum(d[i, j, t, s] for j in range(I) if j != i)
                <= y_plus[i, t, s]
            )
            # 본인이 받는 양은 자신의 y-을 넘을 수 없음
            set.addConstr(
                gp.quicksum(d[j, i, t, s] for j in range(I) if j != i)
                <= y_minus[i, t, s]
            )

# 자기 자신과의 거래 방지
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(d[i, i, t, s] == 0)

# d_given, d_received 정의
for i in range(I):
    for s in range(S):
        set.addConstr(
            d_given[i, s]
            == gp.quicksum(d[i, j, t, s] for j in range(I) if j != i for t in range(T))
        )
        set.addConstr(
            d_received[i, s]
            == gp.quicksum(d[j, i, t, s] for j in range(I) if j != i for t in range(T))
        )

# d+, d- 정의
for i in range(I):
    for s in range(S):
        set.addConstr(d_given[i, s] - d_received[i, s] == d_minus[i, s] - d_plus[i, s])


# d_max_H 정의
for t in range(T):
    for s in range(S):
        for i in range(I):
            set.addConstr(
                d_max_H[t, s] >= gp.quicksum(d[i, j, t, s] for j in range(I) if j != i)
            )

# d_max 제한
for t in range(T):
    for s in range(S):
        set.addConstr(
            d_max_H[t, s] <= 0.3 * gp.quicksum(y_plus[i, t, s] for i in range(I))
        )

# # 수익증가율 정의
# for i in range(I):
#     set.addConstr(profit_increase[i] == (
#         gp.quicksum(
#             P_DA[t] * x[i, t]
#             + 1 / S * gp.quicksum(P_RT[t, s] * e_plus[i, t, s] - P_PN[t] * e_minus[i, t, s] for s in range(S))
#             for t in range(T)
#         ) - only_profit[i].item()
#     ) / only_profit[i].item())

# # 수익증가율이 사람마다 같아야함
# for i in range(I):
#     set.addConstr(
#         profit_increase[i] == gp.quicksum(profit_increase[j] for j in range(I)) / I
#     )

In [22]:
set.optimize()

if set.status == GRB.OPTIMAL:
    x_vals = np.array([[x[i, t].x for t in range(T)] for i in range(I)])
    y_plus_vals = np.array(
        [[[y_plus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    y_minus_vals = np.array(
        [[[y_minus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    d_vals = np.array(
        [
            [[[d[i, j, t, s].x for s in range(S)] for t in range(T)] for j in range(I)]
            for i in range(I)
        ]
    )
    d_given_vals = np.array([d_given[i, s].x for i in range(I) for s in range(S)])
    d_received_vals = np.array([d_received[i, s].x for i in range(I) for s in range(S)])
    e_plus_vals = np.array(
        [[[e_plus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    e_minus_vals = np.array(
        [[[e_minus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    print("\n- - - - - - - - - - - - - - - - - - - - - - - -")
    print("Optimal solution found!")
    print(set.objVal)
else:
    print("\nNo optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
MIPGap  0.001
Heuristics  0.5

Optimize a model with 38250 rows, 55570 columns and 311580 nonzeros
Model fingerprint: 0x7d5b97f1
Variable types: 51610 continuous, 3960 integer (3960 binary)
Coefficient statistics:
  Matrix range     [3e-01, 8e+01]
  Objective range  [5e-04, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-02, 8e+01]
Presolve removed 32306 rows and 42588 columns
Presolve time: 0.13s
Presolved: 5944 rows, 12982 columns, 60481 nonzeros
Variable types: 11909 continuous, 1073 integer (1073 binary)

Root relaxation: objective 1.040156e+04, 20884 iterations, 1.32 seconds (2.20 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 10401.55

### solution pool

In [20]:
print(f"Status: {set.status}")
if set.status == GRB.OPTIMAL:
    print("모델이 Optimal입니다")

    # Solution pool 확인
    print("\nSolution Pool 정보:")
    print(f"찾은 해의 개수: {set.SolCount}")

    # Best solution의 objective value
    best_obj = set.ObjVal
    print(f"\nBest Solution의 Objective Value: {best_obj}")

    # 각 솔루션의 목적함수 값 출력 및 best solution 여부 확인
    solutions = []
    for i in range(set.SolCount):
        set.params.SolutionNumber = i
        solutions.append((i, set.PoolObjVal))
        print(f"\nSolution {i}:")
        print(f"Objective value: {set.PoolObjVal}")
        print(f"Objective bound: {set.PoolObjBound}")
    # Solution pool 파라미터 확인
    print("\nSolution Pool 파라미터:")
    print(f"PoolSolutions: {set.Params.PoolSolutions}")
    print(f"PoolSearchMode: {set.Params.PoolSearchMode}")
    print(f"PoolGap: {set.Params.PoolGap}")
elif set.status == GRB.UNBOUNDED:
    print("모델이 Unbounded입니다")
    # Unbounded ray 확인
    set.computeIIS()
    set.write("model.ilp")
elif set.status == GRB.INFEASIBLE:
    print("모델이 Infeasible입니다")
    # IIS 계산
    set.computeIIS()
    set.write("model.ilp")
elif set.status == GRB.INF_OR_UNBD:
    print("모델이 Infeasible 또는 Unbounded입니다")
    # Presolve를 끄고 다시 시도
    set.setParam("Presolve", 0)
    set.optimize()
elif set.status == GRB.TIME_LIMIT:
    print("시간 제한에 도달했습니다")
    if set.SolCount > 0:
        print(f"현재까지의 최선해: {set.objVal}")
        print(f"Optimality Gap: {set.MIPGap}")

Status: 2
모델이 Optimal입니다

Solution Pool 정보:
찾은 해의 개수: 15

Best Solution의 Objective Value: 10658.39540059959

Solution 0:
Objective value: 10658.395400599575
Objective bound: 10658.39540059937

Solution 1:
Objective value: 10658.395400599517
Objective bound: 10658.39540059937

Solution 2:
Objective value: 10658.395400599504
Objective bound: 10658.39540059937

Solution 3:
Objective value: 10658.395400599453
Objective bound: 10658.39540059937

Solution 4:
Objective value: 10658.395400599431
Objective bound: 10658.39540059937

Solution 5:
Objective value: 10658.395400599398
Objective bound: 10658.39540059937

Solution 6:
Objective value: 10658.395400599395
Objective bound: 10658.39540059937

Solution 7:
Objective value: 10658.395400599395
Objective bound: 10658.39540059937

Solution 8:
Objective value: 10658.395400599391
Objective bound: 10658.39540059937

Solution 9:
Objective value: 10658.395400599378
Objective bound: 10658.39540059937

Solution 10:
Objective value: 10658.395400599378
Ob

In [21]:
# 솔루션 풀에서 해 확인
solution_count = set.SolCount
print(f"총 {solution_count}개의 해가 저장되었습니다.")

for i in range(solution_count):
    set.setParam(GRB.Param.SolutionNumber, i)
    print(f"\n해 {i+1}: 목적 함수 값 = {set.PoolObjVal}")
    # x 변수 값 출력
    for t in range(14, 15):
        for i in range(1, 5):
            print(f"x[{i},{t}] = {x[i, t].Xn}")

총 15개의 해가 저장되었습니다.

해 1: 목적 함수 값 = 10658.395400599575
x[1,14] = 2.5246993142032927
x[2,14] = 4.034638942768362
x[3,14] = 3.096355380169822
x[4,14] = 2.9625922199944306

해 2: 목적 함수 값 = 10658.395400599517
x[1,14] = 2.5246993142032705
x[2,14] = 4.034638942768439
x[3,14] = 3.0963553801698698
x[4,14] = 2.9625922199943693

해 3: 목적 함수 값 = 10658.395400599504
x[1,14] = 2.524699314203313
x[2,14] = 4.0346389427683675
x[3,14] = 3.096355380169789
x[4,14] = 2.9625922199943933

해 4: 목적 함수 값 = 10658.395400599453
x[1,14] = 2.5246993142033025
x[2,14] = 4.034638942768469
x[3,14] = 3.0963553801699257
x[4,14] = 2.9625922199943204

해 5: 목적 함수 값 = 10658.395400599431
x[1,14] = 2.524699314203218
x[2,14] = 4.034638942768371
x[3,14] = 3.0963553801698804
x[4,14] = 2.9625922199944212

해 6: 목적 함수 값 = 10658.395400599398
x[1,14] = 0.0
x[2,14] = 1.8431452654110814
x[3,14] = 3.429049394422039
x[4,14] = 4.37822317171582

해 7: 목적 함수 값 = 10658.395400599395
x[1,14] = 0.0
x[2,14] = 1.8431452654110814
x[3,14] = 3.38574570684

### 결과 분석

#### 수익 분석

In [43]:
# Day-ahead 수익 비교
total_da_profit_obj = 0
for i in range(I):
    for t in range(T):
        total_da_profit_obj += P_DA[t] * x[i, t].x

# Real-time 수익 비교
total_rt_profit_obj = 0
for i in range(I):
    for t in range(T):
        for s in range(S):
            rt_profit_obj = P_RT[t, s] * e_plus[i, t, s].x
            total_rt_profit_obj += 1 / S * rt_profit_obj

# 패널티 비용 비교
total_penalty_cost_obj = 0
for i in range(I):
    for t in range(T):
        for s in range(S):
            penalty_cost_obj = P_PN[t] * e_minus[i, t, s].x
            total_penalty_cost_obj += 1 / S * penalty_cost_obj

# 총 시스템 이익 (목적 함수 기반)
total_system_profit_obj = (
    total_da_profit_obj + total_rt_profit_obj - total_penalty_cost_obj
)

# 결과 출력
print(f"DA: {total_da_profit_obj:.2f}")
print(f"RT: {total_rt_profit_obj:.2f}")
print(f"Penalty: {total_penalty_cost_obj:.2f}")
print(f"총 이익: {total_system_profit_obj:.2f}")

DA: 6673.80
RT: 3941.38
Penalty: 264.16
총 이익: 10351.03


#### daily commitment

In [44]:
for t in range(T):
    avg_e_plus = sum(e_plus[i, t, s].x for i in range(I) for s in range(S)) / S
    avg_e_minus = sum(e_minus[i, t, s].x for i in range(I) for s in range(S)) / S
    print(
        f"[시간 {t}] x: {sum(x[i, t].x for i in range(I)):.3f}, e+: {avg_e_plus:.3f}, e-: {avg_e_minus:.3f}"
    )

total_x = sum(x[i, t].x for i in range(I) for t in range(T))
print(f"총 하루 commitment: {total_x:.3f}")

[시간 0] x: 0.000, e+: 0.000, e-: 0.000
[시간 1] x: 0.000, e+: 0.000, e-: 0.000
[시간 2] x: 0.000, e+: 0.000, e-: 0.000
[시간 3] x: 0.000, e+: 0.000, e-: 0.000
[시간 4] x: 0.000, e+: 0.000, e-: 0.000
[시간 5] x: 0.000, e+: 0.000, e-: 0.000
[시간 6] x: 0.000, e+: 0.000, e-: 0.000
[시간 7] x: 0.000, e+: 0.000, e-: 0.000
[시간 8] x: 0.576, e+: 1.722, e-: 0.077
[시간 9] x: 4.165, e+: 8.369, e-: 0.133
[시간 10] x: 21.486, e+: 11.794, e-: 0.579
[시간 11] x: 30.607, e+: 11.954, e-: 0.597
[시간 12] x: 22.095, e+: 32.216, e-: 0.000
[시간 13] x: 26.993, e+: 24.346, e-: 0.308
[시간 14] x: 32.586, e+: 17.769, e-: 0.181
[시간 15] x: 30.414, e+: 13.732, e-: 1.922
[시간 16] x: 12.118, e+: 9.359, e-: 0.515
[시간 17] x: 1.330, e+: 3.233, e-: 0.061
[시간 18] x: 0.000, e+: 0.402, e-: 0.000
[시간 19] x: 0.000, e+: 0.000, e-: 0.000
[시간 20] x: 0.000, e+: 0.000, e-: 0.000
[시간 21] x: 0.000, e+: 0.000, e-: 0.000
[시간 22] x: 0.000, e+: 0.000, e-: 0.000
[시간 23] x: 0.000, e+: 0.000, e-: 0.000
총 하루 commitment: 182.370


### exchange process

In [45]:
for t in range(12, 14):
    x_sum = sum(x[i, t].x for i in range(I))

    for s in range(S):
        y_plus_sum = sum(y_plus[i, t, s].x for i in range(I))
        y_minus_sum = sum(y_minus[i, t, s].x for i in range(I))

        e_plus_sum = sum(e_plus[i, t, s].x for i in range(I))
        e_minus_sum = sum(e_minus[i, t, s].x for i in range(I))

        print(f"[t={t} s={s}]")

        print(f"x 합계: {x_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {x[i,t].x:.2f}", end=" ")
        print(")")

        print(f"y+ 합계: {y_plus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_plus[i,t,s].x:.2f}", end=" ")
        print(")")

        print(f"y- 합계: {y_minus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_minus[i,t,s].x:.2f}", end=" ")
        print(")")

        print(f"e+ 합계: {e_plus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {e_plus[i,t,s].x:.2f}", end=" ")
        print(")")

        print(f"e- 합계: {e_minus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {e_minus[i,t,s].x:.2f}", end=" ")
        print(")")

        print("거래 내역:")
        for i in range(I):
            for j in range(I):
                if d[i, j, t, s].x > 1e-6:
                    print(f"발전기 {i}가 발전기 {j}에게 {d[i,j,t,s].x:.2f} 전력을 줌")
        print()

[t=12 s=0]
x 합계: 22.10 ([0] 0.00 [1] 0.00 [2] 1.04 [3] 0.00 [4] 9.77 [5] 0.00 [6] 0.00 [7] 11.28 [8] 0.00 [9] 0.00 )
y+ 합계: 49.77 ([0] 8.52 [1] 12.31 [2] 11.86 [3] 6.42 [4] 0.00 [5] 0.00 [6] 0.00 [7] 0.00 [8] 6.72 [9] 3.94 )
y- 합계: 13.00 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 3.70 [5] 0.00 [6] 0.00 [7] 9.30 [8] 0.00 [9] 0.00 )
e+ 합계: 36.77 ([0] 8.52 [1] 4.31 [2] 10.80 [3] 6.42 [4] 0.00 [5] 0.00 [6] 0.00 [7] 0.00 [8] 6.72 [9] 0.00 )
e- 합계: 0.00 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 [5] 0.00 [6] 0.00 [7] 0.00 [8] 0.00 [9] 0.00 )
거래 내역:
발전기 1가 발전기 7에게 8.00 전력을 줌
발전기 2가 발전기 7에게 1.06 전력을 줌
발전기 9가 발전기 4에게 3.70 전력을 줌
발전기 9가 발전기 7에게 0.25 전력을 줌

[t=12 s=1]
x 합계: 22.10 ([0] 0.00 [1] 0.00 [2] 1.04 [3] 0.00 [4] 9.77 [5] 0.00 [6] 0.00 [7] 11.28 [8] 0.00 [9] 0.00 )
y+ 합계: 32.09 ([0] 2.91 [1] 11.02 [2] 0.00 [3] 0.00 [4] 0.00 [5] 4.55 [6] 11.07 [7] 0.00 [8] 0.93 [9] 1.61 )
y- 합계: 13.95 ([0] 0.00 [1] 0.00 [2] 0.53 [3] 0.00 [4] 4.63 [5] 0.00 [6] 0.00 [7] 8.78 [8] 0.00 [9] 0.00 )
e+ 합계: 18.14 (

In [46]:
d_given = np.zeros(I)
d_received = np.zeros(I)

for i in range(I):
    d_given[i] = sum(
        sum(d[i, j, t, s].x for j in range(I) if j != i for t in range(T)) / S
        for s in range(S)
    )
    d_received[i] = sum(
        sum(d[j, i, t, s].x for j in range(I) if j != i for t in range(T)) / S
        for s in range(S)
    )

set_d = pd.DataFrame({"set_d_given": d_given, "set_d_received": d_received})
set_d.to_csv("result/result_set6_d.csv", index=False)

### 정산

In [47]:
total_der_profit = 0
der_profit = {}
for i in range(I):
    der_profit[i] = sum(
        P_DA[t] * x[i, t].x
        + sum(
            1 / S * (P_RT[t, s] * e_plus[i, t, s].x - P_PN[t] * e_minus[i, t, s].x)
            for s in range(S)
        )
        for t in range(T)
    )
    total_der_profit += der_profit[i]

print(f"\n모든 der의 profit 합계: {total_der_profit:.2f}")

print("\n최종 profit:")
for i in range(I):
    print(
        f"[{i}] {der_profit[i]:.2f} ({(der_profit[i]-only_profit[i].item())/only_profit[i].item() * 100:.2f}%)"
    )


모든 der의 profit 합계: 10351.03

최종 profit:
[0] 1218.93 (10.00%)
[1] 910.04 (17.71%)
[2] 1175.68 (1.51%)
[3] 1171.90 (9.15%)
[4] 963.19 (20.51%)
[5] 780.00 (7.64%)
[6] 1319.70 (11.17%)
[7] 1226.36 (13.30%)
[8] 634.75 (0.83%)
[9] 950.48 (8.86%)


### 결과 저장

In [48]:
set_profit_value = pd.DataFrame({'set_profit_value': der_profit})
set_profit_value.to_csv('result/result_set6_profit.csv', index=False)

set_obj = pd.DataFrame({'set_obj': [set.objVal]})
set_obj.to_csv('result/result_set6_obj.csv', index=False)